In [ ]:
#Лабораторная работа №5 Аманжолова А.С.
#Наивный Байесовский классификатор

#Импортируем библиотеки 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Загружаем dataset
import os
print(os.listdir("../input"))

#Читаем CSV-файл
df = pd.read_csv('../input/sms-spam-collection-dataset/spam.csv', encoding='latin-1')[['v1', 'v2']]
df.columns = ['label', 'message']
df.head()

#Опишите набор данных и визуализируйте количество ham/спама
df.groupby('label').describe()

sns.countplot(data=df, x='label')

import string
from nltk.corpus import stopwords
from nltk import PorterStemmer as Stemmer
def process(text):
    # в нижнем регистре
    text = text.lower()
    # убрать знаки препинания
    text = ''.join([t for t in text if t not in string.punctuation])
    # удалить стоп-слова
    text = [t for t in text.split() if t not in stopwords.words('english')]
    # используем алгоритм стемминг
    st = Stemmer()
    text = [st.stem(t) for t in text]
    # возвращает список токенов
    return text

# Тестируем
process('It\'s holiday and we are playing cricket. Jeff is playing very well!!!')

# Тест с нашим набором данных
df['message'][:20].apply(process)

from sklearn.feature_extraction.text import TfidfVectorizer

#Приспособления и преобразования СМС корпуса
tfidfv = TfidfVectorizer(analyzer=process)
data = tfidfv.fit_transform(df['message'])

#Давайте проверим, какие значения он дает для сообщения
mess = df.iloc[2]['message']
print(mess)

print(tfidfv.transform([mess]))

j = tfidfv.transform([mess]).toarray()[0]
print('index\tidf\ttfidf\tterm')
for i in range(len(j)):
    if j[i] != 0:
        print(i, format(tfidfv.idf_[i], '.4f'), format(j[i], '.4f'), tfidfv.get_feature_names()[i],sep='\t')
        
        from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
spam_filter = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer=process)), # сообщения для взвешенной оценки TFIDF
    ('classifier', MultinomialNB())                    # тренировочная выборка на TFIDF векторов с помощью наивного алгоритма Байеса
])

#Выполните функцию train test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.20, random_state = 21)

#Тренировочная спам фильтрация
spam_filter.fit(x_train, y_train)

#Прогнозирование для тестовых случаев
predictions = spam_filter.predict(x_test)

count = 0
for i in range(len(y_test)):
    if y_test.iloc[i] != predictions[i]:
        count += 1
print('Total number of test cases', len(y_test))
print('Number of wrong of predictions', count)

#Проверьте наличие неверных предсказаний, которые были классифицированы как ham
x_test[y_test != predictions]
                 
#Используйте отчет о классификации, чтобы получить более подробную информацию
from sklearn.metrics import classification_report
print(classification_report(predictions, y_test))

def detect_spam(s):
    return spam_filter.predict([s])[0]
detect_spam('Your cash-balance is currently 500 pounds - to maximize your cash-in now, send COLLECT to 83600.')